In [6]:
from sqlalchemy import create_engine, text
import pandas as pd

In [4]:
pg_user = 'postgres'
pg_pass = 'postgres'
pg_host = 'localhost'
pg_port = '5433'
pg_db = 'ny_taxi'

engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')

In [5]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM green_taxi_data LIMIT 5"))
    for row in result:
        print("row:", row)

row: (0, 2, datetime.datetime(2025, 11, 1, 0, 34, 48), datetime.datetime(2025, 11, 1, 0, 41, 39), 'N', 1.0, 74, 42, 1.0, 0.74, 7.2, 1.0, 0.5, 1.94, 0.0, None, 1.0, 11.64, 1.0, 1.0, 0.0, 0.0)
row: (1, 2, datetime.datetime(2025, 11, 1, 0, 18, 52), datetime.datetime(2025, 11, 1, 0, 24, 27), 'N', 1.0, 74, 42, 2.0, 0.95, 7.2, 1.0, 0.5, 0.0, 0.0, None, 1.0, 9.7, 2.0, 1.0, 0.0, 0.0)
row: (2, 2, datetime.datetime(2025, 11, 1, 1, 3, 14), datetime.datetime(2025, 11, 1, 1, 15, 24), 'N', 1.0, 83, 160, 1.0, 2.19, 13.5, 1.0, 0.5, 5.0, 0.0, None, 1.0, 21.0, 1.0, 1.0, 0.0, 0.0)
row: (3, 2, datetime.datetime(2025, 11, 1, 0, 10, 57), datetime.datetime(2025, 11, 1, 0, 24, 53), 'N', 1.0, 166, 127, 1.0, 5.44, 24.7, 1.0, 0.5, 0.5, 0.0, None, 1.0, 27.7, 1.0, 1.0, 0.0, 0.0)
row: (4, 1, datetime.datetime(2025, 11, 1, 0, 3, 48), datetime.datetime(2025, 11, 1, 0, 19, 38), 'N', 1.0, 166, 262, 1.0, 3.2, 18.4, 3.75, 1.5, 1.0, 0.0, None, 1.0, 24.65, 1.0, 1.0, 2.75, 0.0)


In [10]:
df = pd.read_sql(
    con=engine.connect(),
    sql=text("SELECT * FROM green_taxi_data LIMIT 5"),
    parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"]
)

In [11]:
df.head()

,index,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,...,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00,0.0
1,1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,...,0.5,0.00,0.0,None,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,...,0.5,5.00,0.0,None,1.0,21.00,1.0,1.0,0.00,0.0
3,3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,...,0.5,0.50,0.0,None,1.0,27.70,1.0,1.0,0.00,0.0
4,4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,...,1.5,1.00,0.0,None,1.0,24.65,1.0,1.0,2.75,0.0


In [12]:
df.dtypes

index                             int64
VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag                  str
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

# Question 3. Counting short trips

For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?

* 7,853
* 8,007
* 8,254
* 8,421

In [21]:
sql = text("""
    SELECT
        COUNT(*) AS short_trips_november_2025
    FROM green_taxi_data
    WHERE trip_distance <= 1
        AND lpep_pickup_datetime >= '2025-11-01'
        AND lpep_pickup_datetime < '2025-12-01'
""")

df = pd.read_sql(
    con=engine.connect(),
    sql=sql,
    parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"]
)

df

,short_trips_november_2025
0,8007


# 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors).

Use the pick up time for your calculations.

* 2025-11-14
* 2025-11-20
* 2025-11-23
* 2025-11-25

In [27]:
sql = text("""
WITH valid_trips AS (
    SELECT
        lpep_pickup_datetime,
        trip_distance
    FROM green_taxi_data
    WHERE trip_distance < 100
        AND lpep_pickup_datetime >= '2025-11-01'
        AND lpep_pickup_datetime < '2025-12-01'
)

SELECT
    DATE(lpep_pickup_datetime) AS pickup_date
FROM valid_trips
ORDER BY trip_distance DESC
LIMIT 1;
""")

df = pd.read_sql(
    con=engine.connect(),
    sql=sql,
    parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"]
)

df

,pickup_date
0,2025-11-14


# 5. Biggest pickup zone

Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

* East Harlem North
* East Harlem South
* Morningside Heights
* Forest Hills

In [42]:
sql = text("""
WITH total_amount_per_zone AS (
    SELECT
        z."Zone" as zone,
        z."LocationID" as location_id,
        SUM(t.total_amount) AS total_amount
    FROM green_taxi_data t JOIN taxi_zone_lookup z
        ON t."PULocationID" = z."LocationID"
    WHERE t.trip_distance < 100
        AND DATE(t.lpep_pickup_datetime) = '2025-11-18'
    GROUP BY 1, 2
)

SELECT
    *
FROM total_amount_per_zone
ORDER BY total_amount DESC
LIMIT 1;
""")

df = pd.read_sql(
    con=engine.connect(),
    sql=sql,
    parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"]
)

df

,zone,location_id,total_amount
0,East Harlem North,74,9281.92


# Question 6. Largest tip

For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

Note: it's tip , not trip. We need the name of the zone, not the ID.

* JFK Airport
* Yorkville West
* East Harlem North
* LaGuardia Airport

In [51]:
sql = text("""
WITH total_amount_per_zone AS (
    SELECT
        pu_zone."Zone" as pickup_zone,
        do_zone."Zone" as dropoff_zone,
        t.tip_amount
    FROM green_taxi_data t JOIN taxi_zone_lookup pu_zone 
        ON t."PULocationID" = pu_zone."LocationID"
        JOIN taxi_zone_lookup do_zone
        ON t."DOLocationID" = do_zone."LocationID"
    WHERE t.trip_distance < 100
        AND t.lpep_pickup_datetime >= '2025-11-01'
        AND t.lpep_pickup_datetime < '2025-12-01'
        AND pu_zone."Zone" = 'East Harlem North'
)

SELECT
    dropoff_zone AS largest_tip_zone
FROM total_amount_per_zone
ORDER BY tip_amount DESC
LIMIT 1;
""")

df = pd.read_sql(
    con=engine.connect(),
    sql=sql,
    parse_dates=["lpep_pickup_datetime", "lpep_dropoff_datetime"]
)

df

,largest_tip_zone
0,Yorkville West


# 7. Terraform Workflow

Which of the following sequences, respectively, describes the workflow for:

1. Downloading the provider plugins and setting up backend,
2. Generating proposed changes and auto-executing the plan
3. Remove all resources managed by terraform

Answers:

* terraform import, terraform apply -y, terraform destroy
* teraform init, terraform plan -auto-apply, terraform rm
* terraform init, terraform run -auto-approve, terraform destroy
* terraform init, terraform apply -auto-approve, terraform destroy
* terraform import, terraform apply -y, terraform rm

In [53]:
# 1. Downloading the provider plugins and setting up backend,
! terraform init --help

Usage: terraform [global options] init [options]

  Initialize a new or existing Terraform working directory by creating
  initial files, loading any remote state, downloading modules, etc.

  This is the first command that should be run for any new or existing
  Terraform configuration per machine. This sets up all the local data
  necessary to run Terraform that is typically not committed to version
  control.

  This command is always safe to run multiple times. Though subsequent runs
  may give errors, this command will never delete your configuration or
  state. Even so, if you have important information, please back it up prior
  to running this command, just in case.

Options:

  -backend=false          Disable backend or HCP Terraform initialization
                          for this configuration and use what was previously
                          initialized instead.

                          aliases: -cloud=false

  -backend-config=path    Configuration to be merged with 

In [54]:
# 2. Generating proposed changes and auto-executing the plan
! terraform apply -auto-approve --help

Usage: terraform [global options] apply [options] [PLAN]

  Creates or updates infrastructure according to Terraform configuration
  files in the current directory.

  By default, Terraform will generate a new plan and present it for your
  approval before taking any action. You can optionally provide a plan
  file created by a previous call to "terraform plan", in which case
  Terraform will take the actions described in that plan without any
  confirmation prompt.

Options:

  -auto-approve          Skip interactive approval of plan before applying.

  -backup=path           Path to backup the existing state file before
                         modifying. Defaults to the "-state-out" path with
                         ".backup" extension. Set to "-" to disable backup.

  -compact-warnings      If Terraform produces any warnings that are not
                         accompanied by errors, show them in a more compact
                         form that includes only the summary messages

In [55]:
# 3. Remove all resources managed by terraform
! terraform destroy --help

Usage: terraform [global options] destroy [options]

  Destroy Terraform-managed infrastructure.

  This command is a convenience alias for:
      terraform apply -destroy

  This command also accepts many of the plan-customization options accepted by
  the terraform plan command. For more information on those options, run:
      terraform plan -help
